In [2]:
import pickle

In [9]:
with open('../data/q2_submodel_results.pkl') as infile:
    results = pickle.load(infile)

In [6]:
with open('../data/q2_submodel_y_preds.pkl') as infile:
    y_preds = pickle.load(infile)

In [7]:
with open('../data/q2_submodel_y_tests.pkl') as infile:
    y_tests = pickle.load(infile)

In [12]:
results.itervalues().next()

{'best_estimator': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 'best_params': {},
 'best_score': 0.17094545837951813,
 'result':                              0
 mean_fit_time         0.113809
 mean_score_time     0.00805807
 mean_test_score       0.170945
 mean_train_score      0.202891
 params                      {}
 rank_test_score              1
 split0_test_score     0.170945
 split0_train_score    0.202891
 std_fit_time                 0
 std_score_time               0
 std_test_score               0
 std_train_score              0,
 'rmse': 0.78215492084115734}

In [14]:
r = {}

for categ, result in results.iteritems():
    r[categ] = {
        'CV_test': result['best_score'],
        'rmse': result['rmse'],
        'CV_train': result['result'].loc['mean_train_score'].max()
    }

In [52]:
df = pd.DataFrame(r).T
df.head()

,CV_test,CV_train,rmse
Abandoned Bicycle,5.915372e-02,0.149311,0.595545
Abandoned Vehicles,5.945412e-02,0.082168,0.346471
Animal Generic Request,4.533257e-02,0.171777,0.721914
Bed Bugs,-1.460536e+22,0.081804,0.830620
Building Inspection Request,1.438496e-01,0.158578,0.639544


In [53]:
df.sort_values('rmse').head()

,CV_test,CV_train,rmse
Schedule a Bulk Item Pickup SS,0.083546,0.084689,0.294722
Schedule a Bulk Item Pickup,0.100050,0.102385,0.327968
Notification,0.313788,0.393493,0.329748
OCR Front Desk Interactions,0.059783,0.064095,0.335492
Unsanitary Conditions - Establishment,-0.028027,0.077545,0.344100


In [23]:
df.sort_values('CV_test', ascending=False).head()

,CV_test,CV_train,rmse
Recycling Cart Return,0.445682,0.456628,1.084334
Request for Snow Plowing,0.371975,0.373201,0.563758
Notification,0.313788,0.393493,0.329748
Electrical,0.233042,0.358054,0.619165
Needle Pickup,0.207811,0.324483,0.907098


## Looking at coefs for Recycling Cart Return

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship?

## >>>

In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2

In [27]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [28]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(508653, 40)

I'm removing ~1.5% of my rows.

## Choosing columns

In [29]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source', 'neighborhood_from_zip']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [32]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(508653, 31)

## Subsetting by Recycling Cart Return

In [33]:
df = df[df.TYPE == 'Recycling Cart Return']
df.shape

(1986, 31)

## Dummify

In [34]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Property_Type', 'Source', 'neighborhood_from_zip', 'school', 'housing']

In [35]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Intersection is baseline 0 5
Constituent Call is baseline 1 5
West Roxbury is baseline 2 5
8_6th_grade is baseline 3 5
rent is baseline 4 5


In [36]:
df_dummified.shape

(1986, 54)

## Running model

In [48]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [38]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [42]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [46]:
col_list = ' + '.join(df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1))

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     COMPLETION_HOURS_LOG_10   R-squared:                       0.458
Model:                                 OLS   Adj. R-squared:                  0.441
Method:                      Least Squares   F-statistic:                     26.52
Date:                     Wed, 22 Feb 2017   Prob (F-statistic):          2.49e-168
Time:                             07:36:43   Log-Likelihood:                -2456.4
No. Observations:                     1588   AIC:                             5013.
Df Residuals:                         1538   BIC:                             5281.
Df Model:                               49                                         
Covariance Type:                 nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                                 1.5704      0.664      2.366      0.018         0.269     2.872
SubmittedPhoto[T.True]                                 5.199e-10   2.47e-10      2.105      0.035      3.54e-11     1e-09
isdescription[T.True]                                 -6.839e-11   6.11e-11     -1.120      0.263     -1.88e-10  5.14e-11
racewhite                                                 0.1647      0.241      0.685      0.494        -0.307     0.637
raceblack                                                 0.2899      0.238      1.216      0.224        -0.178     0.757
raceasian                                                -0.2347      0.317     -0.740      0.460        -0.857     0.388
racehispanic                                              0.6039      0.305      1.981      0.048         0.006     1.202
raceother                                                 0.7467      0.476      1.567      0.117        -0.188     1.681
povertypopbelowpovertylevel                              -0.3773      0.463     -0.814      0.416        -1.286     0.531
earnedincomepercapita                                  4.347e-06    4.2e-06      1.034      0.301      -3.9e-06  1.26e-05
povertypopwpublicassistance                               0.2450      0.678      0.361      0.718        -1.085     1.575
povertypopwfoodstamps                                    -0.3510      0.422     -0.831      0.406        -1.179     0.477
povertypopwssi                                            2.1198      0.542      3.913      0.000         1.057     3.182
schoolstddev                                              0.0006      0.002      0.288      0.774        -0.004     0.005
housingstddev                                             0.0010      0.000      2.131      0.033       8.2e-05     0.002
bedroom                                                   0.0551      0.049      1.118      0.264        -0.042     0.152
bedroomstddev                                            -0.0029      0.002     -1.762      0.078        -0.006     0.000
value                                                  -2.58e-07    1.8e-07     -1.435      0.152     -6.11e-07  9.47e-08
valuestddev                                              -0.0109      0.006     -1.679      0.093        -0.024     0.002
rent                                                  -5.506e-06   6.83e-05     -0.081      0.936        -0.000     0.000
rentstddev                                                0.0112      0.005      2.124      0.034         0.001     0.022
income                                                 3.178e-08   6.66e-07      0.048      0.962   

## Interpreting model

Which features are most associated with completion time?

In [49]:
df = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df.columns][1:]
df['coef_abs'] = pd.np.abs(df.coef)
df.head()

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept ...,1.570400e+00,6.640000e-01,2.366,0.018,0.269 2.872,1.570400e+00
1,SubmittedPhoto[T.True] ...,5.199000e-10,2.470000e-10,2.105,0.035,3.54e-11 1e-09,5.199000e-10
2,isdescription[T.True] ...,-6.839000e-11,6.110000e-11,-1.120,0.263,-1.88e-10 5.14e-11,6.839000e-11
3,racewhite ...,1.647000e-01,2.410000e-01,0.685,0.494,-0.307 0.637,1.647000e-01
4,raceblack ...,2.899000e-01,2.380000e-01,1.216,0.224,-0.178 0.757,2.899000e-01


In [50]:
df.sort_values('P>|t|')

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
33,neighborhoodfromzipEastBoston ...,-1.335000e+00,2.810000e-01,-4.757,0.000,-1.885 -0.785,1.335000e+00
31,neighborhoodfromzipDorchester ...,-1.025600e+00,2.040000e-01,-5.032,0.000,-1.425 -0.626,1.025600e+00
24,queuewkopen ...,-7.200000e-03,0.000000e+00,-18.067,0.000,-0.008 -0.006,7.200000e-03
23,queuewk ...,4.000000e-04,2.000000e-05,18.334,0.000,0.000 0.000,4.000000e-04
12,povertypopwssi ...,2.119800e+00,5.420000e-01,3.913,0.000,1.057 3.182,2.119800e+00
37,neighborhoodfromzipMattapan ...,-7.032000e-01,2.360000e-01,-2.979,0.003,-1.166 -0.240,7.032000e-01
34,neighborhoodfromzipFenwayKenmoreAudubonCircleL...,-1.352000e+00,4.680000e-01,-2.891,0.004,-2.269 -0.435,1.352000e+00
26,neighborhoodfromzipAllstonBrighton ...,-6.326000e-01,2.470000e-01,-2.559,0.011,-1.117 -0.148,6.326000e-01
35,neighborhoodfromzipHydePark ...,-6.176000e-01,2.460000e-01,-2.514,0.012,-1.099 -0.136,6.176000e-01
41,neighborhoodfromzipRoxbury ...,-8.029000e-01,3.380000e-01,-2.376,0.018,-1.466 -0.140,8.029000e-01


## Conclusion

Making sub-models improves the R2 and RMSE _for these chosen categories_, which are the ones above the 70th percentile in terms of number of issues, minus a couple ones stated above that messed up the R2.

This R2 and RMSE are 0.59 and 0.69. The ones for the main model are 0.55 and 0.73.

There are prolly more stat sig coefs here as well.

## Next Steps

If I had more time,
- check how the R2 was messed up on those couple of categs. Prolly bc R2 for that model was low, and my preds were way off. what to do about those categs then? either the mean, or the pred from the big model would work.
- group the categs below 70th percentile to give them enough sample size, then run model on them. trial-and-error + domain knowledge as to which categs would work and which wouldn't.